In [6]:
import tensorflow as tf
filename_queue = tf.train.string_input_producer(
    tf.train.match_filenames_once('~/pooh/poohdrop/mldata/tf_inteligence/output/training-images/*.tfrecords'))
reader = tf.TFRecordReader()
_, serialized = reader.read(filename_queue)
features = tf.parse_single_example(
    serialized,
    features={
        'label': tf.FixedLenFeature([], tf.string),
        'image': tf.FixedLenFeature([], tf.string),
    })
record_image = tf.decode_raw(features['image'], tf.uint8)
record_image

<tf.Tensor 'DecodeRaw_1:0' shape=(?,) dtype=uint8>

In [7]:
image = tf.reshape(record_image, [250, 151, 1])
label = tf.cast(features['label'], tf.string)
min_after_dequeue = 10
batch_size = 3
capacity = min_after_dequeue + 3 * batch_size
image_batch, label_batch = tf.train.shuffle_batch(
    [image, label], batch_size=batch_size, capacity=capacity,
    min_after_dequeue=min_after_dequeue)
image_batch

<tf.Tensor 'shuffle_batch:0' shape=(3, 250, 151, 1) dtype=uint8>

In [58]:
# Converting the images to a float of [0,1) to match the expected input to convolution2d
float_image_batch = tf.image.convert_image_dtype(image_batch, tf.float32)
conv2d_layer_one = tf.contrib.layers.convolution2d(
    float_image_batch,
    num_outputs=32,
    kernel_size=(5,5),
    activation_fn=tf.nn.relu,
    weights_initializer=tf.random_normal_initializer(),
    stride=(2,2),
    trainable=True)
pool_layer_one = tf.nn.max_pool(conv2d_layer_one,
                                ksize=[1,2,2,1],
                               strides=[1,2,2,1],
                               padding='SAME')

In [59]:
# Note, the first and last dimension of the convolution output hasn't changed but the
# middle two dimensions have
conv2d_layer_one.get_shape(), pool_layer_one.get_shape()

(TensorShape([Dimension(3), Dimension(125), Dimension(76), Dimension(32)]),
 TensorShape([Dimension(3), Dimension(63), Dimension(38), Dimension(32)]))